<a href="https://colab.research.google.com/github/SatoJin02/BDA_course25/blob/main/ex02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import pandas as pd
import plotly.express as px

df_city_population = pd.read_csv("./city_population.csv")

fig = px.scatter_mapbox(
    # データフレームおよび緯度・経度の設定
    data_frame=df_city_population,
    lat="latitude",
    lon="longitude",
    # ホバー時の表示設定
    hover_name="city",
    hover_data=["population"],
    # 散布図のマーカーの設定
    color="population",
    size="population",
    size_max=30,
    opacity=0.4,
    # 作図の設定
    center={'lat':34.686567, 'lon':135.52000},
    zoom=4,
    height=600,
    width=800)
fig.update_layout(mapbox_style='open-street-map')
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.update_layout(title_text="Population of each cities")
fig.show()
